# Анализ сервиса вопросов и ответов по программированию

**Описание проекта**

В исходных данных — база данных StackOverflow. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам. В базе хранятся данные о постах за 2008 год, но в таблицах также информация и о более поздних оценках, которые эти посты получили.

Задача — с помощью SQL необходимо посчитать ключевые метрики сервис-системы вопросов и ответов о программировании.

**Описание данных**

ER-диаграмма базы:

<img src="https://pictures.s3.yandex.net/resources/Frame_353_1_1664969703.png" width="600" align="center">

Структура `stackoverflow.badges` — хранит информацию о значках, которые присуждаются за разные достижения:
* id — идентификатор значка, первичный ключ таблицы;
* name	— название значка;
* user_id — идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users;
* creation_date	— дата присвоения значка.

Структура таблицы `stackoverflow.post_types` — cодержит информацию о типе постов (Question — пост с вопросом; Answer — пост с ответом):
* id	— идентификатор типа поста, первичный ключ таблицы;
* type	— тип поста.

Структура таблицы `stackoverflow.posts` — cодержит информацию о постах:
* id	— идентификатор поста, первичный ключ таблицы;
* title	— заголовок поста;
* creation_date	— дата создания поста;
* favorites_count	— число, которое показывает, сколько раз пост добавили в «Закладки»;
* last_activity_date	— дата последнего действия в посте, например комментария;
* last_edit_date	— дата последнего изменения поста;
* user_id	— идентификатор пользователя, который создал пост, внешний ключ к таблице users;
* parent_id	— если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом;
* post_type_id	— идентификатор типа поста, внешний ключ к таблице post_types;
* score	— количество очков, которое набрал пост;
* views_count	— количество просмотров.

Структура таблицы `stackoverflow.users` — содержит информацию о пользователях:
* id	— идентификатор пользователя, первичный ключ таблицы;
* creation_date	— дата регистрации пользователя;
* display_name	— имя пользователя;
* last_access_date	— дата последнего входа;
* location	— местоположение;
* reputation	— очки репутации, которые получают за хорошие вопросы и полезные ответы;
* views	— число просмотров профиля пользователя.

Структура таблицы `stackoverflow.vote_types` — содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту:
* id	— идентификатор типа голоса, первичный ключ;
* name	— название метки.

Структура таблицы `stackoverflow.votes` — cодержит информацию о голосах за посты:
* id	— идентификатор голоса, первичный ключ;
* post_id	— идентификатор поста, внешний ключ к таблице posts;
* user_id	— идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users;
* bounty_amount	— сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту;
* vote_type_id	— идентификатор типа голоса, внешний ключ к таблице vote_types;
* creation_date	— дата назначения голоса.

## Общая информация о данных:

Импортируем необходмые библиотеки:

In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text, create_engine

Подключимся к базе данных:

In [3]:
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраним коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# напишем функцию для выполнения SQL-запросоов
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

Выведем датафрейм `stackoverflow.posts`:


In [4]:
db = '''
SELECT *
FROM stackoverflow.posts
'''
get_sql_data(db).head(5)

,id,title,creation_date,favorites_count,last_activity_date,last_edit_date,user_id,parent_id,post_type_id,score,views_count
0,4,Convert Decimal to Double?,2008-07-31 21:42:53,41,2018-07-02 17:55:27.247,2018-07-02 17:55:27,8,0,1,573,37080
1,6,Percentage width child element in absolutely p...,2008-07-31 22:08:09,10,2016-03-19 06:10:52.170,2016-03-19 06:05:48,9,0,1,256,16306
2,7,None,2008-07-31 22:17:58,0,2017-12-16 05:06:57.613,2017-12-16 05:06:58,9,4,2,401,0
3,9,How do I calculate someone's age in C#?,2008-07-31 23:41:00,399,2018-07-25 11:57:14.110,2018-04-21 17:48:14,1,0,1,1743,480476
4,11,Calculate relative time in C#,2008-07-31 23:55:38,529,2018-07-05 04:00:56.633,2017-06-04 15:51:20,1,0,1,1348,136033


**Задание 1.** Найдем количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [5]:
task_one = '''
SELECT COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id=1
  AND (score>300 OR favorites_count >= 100)
GROUP BY post_type_id;
'''
get_sql_data(task_one).head(5)

,count
0,1355



**Задание 2.** Найдем среднее количество вопросов с 1 по 18 ноября 2008 включительно.

In [6]:
task_two = '''
WITH
-- подзапрос
i AS (SELECT COUNT(id) AS total,
             creation_date::date
      FROM stackoverflow.posts
      WHERE post_type_id = 1
      GROUP BY creation_date::date
      HAVING creation_date::date BETWEEN '2008-11-01' AND '2008-11-18')
   -- основной запрос
SELECT ROUND(AVG(total), 0)
FROM i;
'''
get_sql_data(task_two).head(5)

,round
0,383.0



**Задание 3.** Найдем количество пользователей, которые получили значки сразу в день регистрации. Выведем количество уникальных пользователей.

In [7]:
task_three = '''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u ON b.user_id=u.id
WHERE b.creation_date::date = u.creation_date::date;
'''
get_sql_data(task_three).head(5)

,count
0,7047


**Задание 4.** Найдем количество уникальных постов пользователя с именем Joel Coehoorn, которые получили хотя бы один голос?

In [8]:
task_four = '''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u ON b.user_id=u.id
WHERE b.creation_date::date = u.creation_date::date;
'''
get_sql_data(task_four).head(5)

,count
0,7047


**Задание 5.** Выгрузим все поля таблицы vote_types. Добавим к таблице поле rank, в которое войдут номера записей в обратном порядке. Отсортируем таблицу по полю id.

In [9]:
task_five = '''
SELECT *,
       ROW_NUMBER() OVER(ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;
'''
get_sql_data(task_five).head(5)

,id,name,rank
0,1,AcceptedByOriginator,15
1,2,UpMod,14
2,3,DownMod,13
3,4,Offensive,12
4,5,Favorite,11


**Задание 6.** Отберем 10 пользователей, которые поставили больше всего голосов типа Close. Отобразим таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируем данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [10]:
task_six = '''
SELECT sv.user_id,
       COUNT(svt.name) AS cnt
FROM stackoverflow.votes AS sv
LEFT JOIN stackoverflow.vote_types AS svt ON sv.vote_type_id = svt.id
WHERE svt.name = 'Close'
GROUP BY sv.user_id
ORDER BY cnt DESC, sv.user_id DESC
LIMIT 10
'''
get_sql_data(task_six).head(10)

,user_id,cnt
0,20646,36
1,14728,36
2,27163,29
3,41158,24
4,24820,23
5,9345,23
6,3241,23
7,44330,20
8,38426,19
9,19074,19


**Задание 7.** Отберем 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразим несколько полей:

* идентификатор пользователя;

* число значков;

* место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвоим одно и то же место в рейтинге.
Отсортируем записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [11]:
task_seven = '''
WITH
-- подзапрос для среза по дате
i AS (SELECT user_id,
             COUNT(id) AS cnt
      FROM stackoverflow.badges
      WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
      GROUP BY user_id
      ORDER BY cnt DESC, user_id ASC)
-- основной запрос
SELECT *,
       DENSE_RANK() OVER (ORDER BY cnt DESC) AS rn
FROM i
LIMIT 10
'''
get_sql_data(task_seven).head(5)

,user_id,cnt,rn
0,22656,149,1
1,34509,45,2
2,1288,40,3
3,5190,31,4
4,13913,30,5


**Задание 8.** Определим сколько в среднем очков получает пост каждого пользователя - сформируем таблицу из следующих полей:

* заголовок поста;

* идентификатор пользователя;

* число очков поста;

* среднее число очков пользователя за пост, округлённое до целого числа.

Посты без заголовка, а также те, что набрали ноль очков, не учитываются.

In [12]:
task_eight = '''
WITH
-- подзапрос 1
i AS (SELECT ROUND(AVG(score)) AS avg_score,
             user_id
      FROM stackoverflow.posts
      WHERE title IS NOT NULL
        AND score != 0
      GROUP BY user_id)

SELECT p.title,
       i.user_id,
       p.score,
       i.avg_score
FROM i
JOIN stackoverflow.posts AS p ON i.user_id = p.user_id
WHERE p.title IS NOT NULL
   AND p.score != 0;
'''
get_sql_data(task_eight).head(5)

,title,user_id,score,avg_score
0,Convert Decimal to Double?,8,573,573.0
1,Percentage width child element in absolutely p...,9,256,200.0
2,How do I calculate someone's age in C#?,1,1743,573.0
3,Calculate relative time in C#,1,1348,573.0
4,Determine a User's Timezone,9,539,200.0


**Задание 9.** Отобразим заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не учитываются в списке.

In [13]:
task_nine = '''
WITH
-- подзапрос
i AS (SELECT user_id,
             COUNT(id) AS cnt
      FROM stackoverflow.badges
      GROUP BY user_id
      HAVING COUNT(id) > 1000)
-- основной запрос
SELECT title
FROM i
JOIN stackoverflow.posts AS p ON i.user_id = p.user_id
WHERE p.title IS NOT NULL

'''
get_sql_data(task_nine).head(5)

,title
0,What's the strangest corner case you've seen i...
1,What's the hardest or most misunderstood aspec...
2,What are the correct version numbers for C#?
3,Project management to go with GitHub


**Задание 10.** Напишем запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделим пользователей на три группы в зависимости от количества просмотров их профилей:

* пользователяс с числом просмотров больше либо равным 350 присвоим группу 1;

* пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;

* пользователям с числом просмотров меньше 100 — группу 3.

Отобразим в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны вошли в итоговую таблицу.

In [14]:
task_ten = '''
SELECT id,
       views,
       CASE
          WHEN views>=350 THEN 1
          WHEN views<100 THEN 3
          ELSE 2
       END AS group
FROM stackoverflow.users
WHERE location LIKE '%Canada%'
   AND views > 0;
'''
get_sql_data(task_ten).head(5)

,id,views,group
0,22,1079,1
1,34,1707,1
2,37,757,1
3,41,174,2
4,42,1063,1


**Задание 11.** Дополним предыдущий запрос. Отобразим лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведим поля с идентификатором пользователя, группой и количеством просмотров. Отсортируем таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [15]:
task_eleven = '''
WITH
-- подзапрос 1
i AS (SELECT id,
             views,
             CASE
                 WHEN views>=350 THEN 1
                 WHEN views<100 THEN 3
             ELSE 2
             END AS group
       FROM stackoverflow.users
       WHERE location LIKE '%Canada%'
         AND views > 0),
-- подзапрос 2
m AS (SELECT *,
             MAX(i.views) OVER(PARTITION BY i.group) AS max_views
      FROM i)
-- основной запрос
SELECT m.id,
       m.group,
       m.views
FROM m
WHERE m.views = m.max_views
ORDER BY m.max_views DESC, id ASC
'''
get_sql_data(task_eleven).head(5)

,id,group,views
0,3153,1,21991
1,46981,2,349
2,3444,3,99
3,22273,3,99
4,190298,3,99


**Задание 12.** Посчитаем ежедневный прирост новых пользователей в ноябре 2008 года. Сформируем таблицу с полями:

* номер дня;

* число пользователей, зарегистрированных в этот день;

* сумму пользователей с накоплением.

In [16]:
task_twelve= '''
WITH
-- подзапрос
i AS (SELECT EXTRACT(DAY FROM creation_date::date) AS days,
             COUNT(id) AS cnt
      FROM stackoverflow.users
      WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
      GROUP BY EXTRACT(DAY FROM creation_date::date))

SELECT *,
       SUM(cnt) OVER (ORDER BY days)
FROM i
'''
get_sql_data(task_twelve).head(5)

,days,cnt,sum
0,1.0,34,34.0
1,2.0,48,82.0
2,3.0,75,157.0
3,4.0,192,349.0
4,5.0,122,471.0


**Задание 13.** Для каждого пользователя, который написал хотя бы один пост, найдем интервал между регистрацией и временем создания первого поста.

Отобразим:

* идентификатор пользователя;

* разницу во времени между регистрацией и первым постом.

In [17]:
task_thirteen = '''
WITH
i AS (SELECT DISTINCT user_id,
             MIN(creation_date) OVER (PARTITION BY user_id) AS first_post
 FROM stackoverflow.posts)

SELECT user_id,
       i.first_post - u.creation_date AS diff
FROM i
JOIN stackoverflow.users AS u ON i.user_id = u.id
'''
get_sql_data(task_thirteen).head(5)

,user_id,diff
0,41040,0 days 00:20:34
1,18721,0 days 00:17:46
2,279,2 days 00:07:41
3,19068,1 days 01:54:55
4,12318,0 days 00:00:00


**Задание 14.** Выведем общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируем по убыванию общего количества просмотров.

In [18]:
task_fourteen = '''
SELECT SUM(views_count),
       DATE_TRUNC('month', creation_date)::date AS mnth
FROM stackoverflow.posts
WHERE DATE_TRUNC('year', creation_date)::date = '2008-01-01'
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY SUM(views_count) DESC;
'''
get_sql_data(task_fourteen).head(5)

,sum,mnth
0,452928568,2008-09-01
1,365400138,2008-10-01
2,221759651,2008-11-01
3,197792841,2008-12-01
4,131367083,2008-08-01


**Задание 15.** Выведем имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывались. Для каждого имени пользователя выведем количество уникальных значений user_id. Отсортируем результат по полю с именами в лексикографическом порядке.

In [19]:
task_fifteen = '''
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
JOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id
WHERE p.creation_date::date BETWEEN u.creation_date::date AND (u.creation_date::date + INTERVAL '1 month')
   AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY u.display_name;
'''
get_sql_data(task_fifteen).head(5)

,display_name,count
0,1800 INFORMATION,1
1,Adam Bellaire,1
2,Adam Davis,1
3,Adam Liss,1
4,Alan,8


**Задание 16.** Выведем количество постов за 2008 год по месяцам. Отберем посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируем таблицу по значению месяца по убыванию.

In [20]:
task_sixteen = '''
WITH t AS (
           SELECT u.id
           FROM stackoverflow.posts AS p
           JOIN stackoverflow.users AS u ON p.user_id = u.id
           WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01'
              AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'
           GROUP BY u.id
           HAVING COUNT(p.id)>0
          )
SELECT COUNT(p.id),
       DATE_TRUNC('month', p.creation_date)::date
FROM stackoverflow.posts AS p
WHERE p.user_id IN (SELECT * FROM t)
   AND DATE_TRUNC('year', p.creation_date)::date = '2008-01-01'
GROUP BY DATE_TRUNC('month', p.creation_date)::date
ORDER BY DATE_TRUNC('month', p.creation_date)::date DESC;
'''
get_sql_data(task_sixteen).head(5)

,count,date_trunc
0,17641,2008-12-01
1,18294,2008-11-01
2,27171,2008-10-01
3,24870,2008-09-01
4,32,2008-08-01


**Задание 17.** Используя данные о постах, выведем несколько полей:

* идентификатор пользователя, который написал пост;

* дата создания поста;

* количество просмотров у текущего поста;

* сумма просмотров постов автора с накоплением.

Данные в таблице отсортируем по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [21]:
task_seventeen = '''
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts;
'''
get_sql_data(task_seventeen).head(5)

,user_id,creation_date,views_count,sum
0,1,2008-07-31 23:41:00,480476,480476
1,1,2008-07-31 23:55:38,136033,616509
2,1,2008-07-31 23:56:41,0,616509
3,1,2008-08-04 02:45:08,0,616509
4,1,2008-08-04 04:31:03,0,616509


**Задание 18.** Найдем сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой. Для каждого пользователя отберем дни, в которые он или она опубликовали хотя бы один пост.

In [22]:
task_eighteen = '''
SELECT ROUND(AVG(t.cnt))
FROM (
      SELECT user_id,
             COUNT(DISTINCT creation_date::date)  AS cnt
      FROM stackoverflow.posts
      WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
      GROUP BY user_id
) AS t
'''
get_sql_data(task_eighteen).head(5)

,round
0,2.0


**Задание 19.** Найдем на сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразим таблицу со следующими полями:

* Номер месяца.

* Количество постов за месяц.

* Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлим значение процента до двух знаков после запятой.

In [23]:
task_nineteen = '''
WITH t AS (
SELECT EXTRACT(MONTH from creation_date::date) AS month,
       COUNT(DISTINCT id)
FROM stackoverflow.posts
WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY month
)

SELECT *,
       ROUND(((count::numeric / LAG(count) OVER (ORDER BY month)) - 1) * 100,2) AS user_growth
FROM t;
'''
get_sql_data(task_nineteen).head(5)

,month,count,user_growth
0,9.0,70371,NaN
1,10.0,63102,-10.33
2,11.0,46975,-25.56
3,12.0,44592,-5.07


**Задание 20.** найдем пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведем данные его активности за октябрь 2008 года в таком виде:

* номер недели;

* дата и время последнего поста, опубликованного на этой неделе.

In [24]:
task_twenty = '''
WITH
-- предзапрос 1
t AS (SELECT user_id,
             COUNT(DISTINCT id) AS cnt
      FROM stackoverflow.posts
      GROUP BY user_id
      ORDER BY cnt DESC
      LIMIT 1),
-- предзапрос 2
t1 AS (SELECT p.user_id,
              p.creation_date,
              EXTRACT('week' from p.creation_date) AS week_number
       FROM stackoverflow.posts AS p
       JOIN t ON t.user_id = p.user_id
       WHERE DATE_TRUNC('month', p.creation_date)::date = '2008-10-01')
-- основной запрос
SELECT DISTINCT week_number::numeric,
       MAX(creation_date) OVER (PARTITION BY week_number)
FROM t1
ORDER BY week_number;
'''
get_sql_data(task_twenty).head(5)

,week_number,max
0,40.0,2008-10-05 09:00:58
1,41.0,2008-10-12 21:22:23
2,42.0,2008-10-19 06:49:30
3,43.0,2008-10-26 21:44:36
4,44.0,2008-10-31 22:16:01
